# Hound: Locating Cryptographic Primitives in Desynchronized Side-Channel Traces Using Deep Learning

In [1]:
import numpy as np

from CNN.train import train 
from inference_pipeline.sliding_window_classification import getModule, classifyTrace, saveClassification
from inference_pipeline.screening import *
from inference_pipeline.alignment import alignCps, saveCps
from inference_pipeline.debug import *
from matplotlib import pyplot as plt

## Train CNN

Each CNN is configure thanks to a YALM configuration file.  
You can set different module hyper-parameters as well as the dataset, the logger, and the experiment configurations.  
Default configuration are in `CNN/configs` directory, both for Neputune logger and for the experiment. 

In [ ]:
config_folder = "CNN/configs/exp/" # /path/to/experiment/config/folder/
train(config_folder)

## Sliding Window Classification

In [ ]:
SID = "<Neptune_SID>"
trace_file = "</path/to/trace_file>"
classification_output_file = "</path/to/output_file>"

Get the trained CNN starting from Neptune SID.

In [ ]:
module = getModule(SID)

Compute and save segmentation as file `output_file`.  
Function _classifyTrace_ has a few parameters to configure based on the experiment:
- `stride`: Define the stride to use for the sliding window.
- `window_size`: Define the size of the sliding window itself.

In [4]:
window_size = 10_000
stride = 62

In [ ]:
classifications = classifyTrace(trace_file, module, stride, window_size, gpu=1)
saveClassification(classifications, classification_output_file)

## Screening

Screen the classification output and find the starting sample of each cryptographic primitive.

In [36]:
stride = 62 # same as input of Sliding Window Classification
classification_file =  "</path/to/classification_output_file>"
gt_file = "</path/to/gt_file>"

gt = np.load(gt_file)
classifications = np.load(classification_file, mmap_mode='r')

avg_cp_lenght = 145_000
major_filter_size = 150

In [ ]:
CPs = []
for classification in classifications:
    CPs.append(segment(classification, major_filter_size, stride, avg_cp_lenght))

fp, fn = errorCount(gt[0], np.asarray(CPs[0]), stride)
print(f"Errors: {fp+fn} ({fn} false negatives, {fp} false positives)")

### Visualization

Visualize the detected CP and ground truth (GT) for a given trace.

**First Subplot**: Draws black vertical lines for detected CP and red dashed vertical lines for GT.  
**Second Subplot**: Plots classification for each point in the trace.

In [ ]:
n_trace = 0
center = 100_000
margin = 15_000
lim = (center-margin, center+margin)

fig, ax = plt.subplots(2, figsize=(15, 5))

ax[0].set_xlim(lim)
# Plot vertical lines for found CPs
for sample in CPs[n_trace][1:]:
    ax[0].axvline(x=sample, color='black', linestyle='-')
ax[0].axvline(x=CPs[n_trace][0], color='black', linestyle='-', label='Found CP')
# Plot vertical lines for gt
for sample in gt[n_trace, 1:]:
    ax[0].axvline(x=sample//stride, color='r', linestyle='--')
ax[0].axvline(x=gt[n_trace, 0]//stride, color='r', linestyle='--', label='Ground Truth')
ax[0].legend(loc='lower right', bbox_to_anchor=(1, 0.95))

ax[1].set_xlim(lim)
ax[1].plot(np.argmax(classifications[n_trace], axis=1))

## Alignment

Align in time the found CPs.  
Some parameters, such as _slide_, has to be set as used in sliding window classification.

In [ ]:
trace_file =  "</path/to/trace_file>" # same as input of Sliding Window Classification
aligned_output_file = "</path/to/output_file>"
stride = 62 # same as input of Sliding Window Classification

In [ ]:
trace =  np.load(trace_file)
alligned = alignCps(trace, CPs, stride)
saveCps(alligned, aligned_output_file)